This scorecard contains multigig-enablement data at the State level, including:
Current XGSPON enablement - from LU QUAL
Current Multi-gig enablement - from LU QUAL (with hardcoded polygon id's)
Next month construction funded XGS-PON LUs - from PowerBI XGS Prep dashboard
Next month construction funded LUs with core network = Multigig

## Please do a fresh import for this report:
##### 1. Construction funding info to the end of the year from PowerBI - do not pull again until 2023
##### 2. Current copy of the summarized salesforce report  (because it's already been aggregated)

## Information needed from the engineers:
##### 1. Speedtest server ready?
##### 2. Out of MG & construction funded, how many LU's require additional 100G speedtest servers to support mg
##### 3. Final manual WC exclusions from mg opportunities list with sub info view

In [1]:
import os
import sys
import pandas as pd
import numpy as np
import cx_Oracle
cx_Oracle.init_oracle_client(lib_dir=r"C:\Oracle\instantclient_21_6")

### Queries

In [2]:
# FRAM001P
conn = cx_Oracle.connect(
    user="AD21236",
    password="L2022$Um",
    dsn="RACORAP08-SCAN.CORP.INTRANET/FRAM001P_PROD",
    encoding="UTF-8")

In [3]:
# XGS qualifications count at wire center level
# 10/18/22 - add OLT_NET_ID to data pull

query = """
            WITH QD AS (
            SELECT  
                    O.LUID --count distinct LU id's
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,count(distinct LUID) as xgs_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
            

                                """

query2 = """
            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_POLYGONID
                    ,O.UNI_SERVICETYPE
                    ,O.UNI_STATUS
                    ,O.TECH_TYPE
                    ,O.FSAI
                    ,O.OLT_TYPE
                    ,O.ADDRESS
                    ,O.OLT
                    ,O.UNI_ECD
                    ,SUBSTR(O.OLT_NET_ID,1,11) as OLT_NET_ID   --10/18/22 added per Joe advise
                    --O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  *
                    
            FROM    QD

                                """

df_ora = pd.read_sql(query, con=conn)
df_ora_details = pd.read_sql(query2, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [4]:
%%time

# count of MG enabled LUs at wire center level

mg_query = """

            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in ('1179499',  -- PHNXAZNE
                                          '1179547', '1179498', '1179121','1180067','1180494', -- PHNXAZSO
                                          '1178832', -- ARVDCOMA
                                          '1179849','1180344','1180107','1180306','1180028', '1178694', '1179569',-- MPLSMNBE 
                                          '1179862', '1179836', '1179604', '1179838', '1179887', '1180054', '1180222', -- MPLSMNBE
                                          '1178144', --PYLPWA01
                                          '1179687', '1179674','1180254', --GLDNCOMA
                                          '1180012','1180391', -- PTLDOR11
                                          '1179864', -- DNVRCOMA 
                                          '1179553','1179924', -- STPLMNBE 
                                          '1179688','1179157', '1179574',  -- MPLSMNTF
                                          '1180026','1180288','1179808','1179976', -- MPLSMNGE
                                          '1179583', -- DNVRCODC
                                          '1179774','1180265','1180031','1180237', -- CRYSMNCR
                                          '1180315','1189074', -- MRRYUTMA
                                          '1179708','1179714','1179719','1180072','1180075','1180089','1180095', -- TCSNAZMA
                                          '1180096','1179639','1179689','1179690','1179700','1179701','1179702', -- TCSNAZMA
                                          '1179703','1179704','1179718','1179793','1179803','1180092','550715', -- TCSNAZMA
                                          '1180094','1180507', --TCSNAZMA
                                          '1179641','1179914' --TCSNAZCR
                                          
                                          ) --add in MG polygon IDs here
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,count(distinct LUID) as mg_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID


"""

mg_lu_ora = pd.read_sql(mg_query, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


CPU times: total: 0 ns
Wall time: 803 ms


In [5]:
# check data
print(mg_lu_ora.head())
print(df_ora.head())

  STATE PRIMARY_WIRE_CENTER_ID  MG_ENABLED_CNT
0    CO               GLDNCOMA             412
1    CO               DNVRCODC             166
2    AZ               TCSNAZMA            8203
3    WA               PYLPWA01             340
4    CO               ARVDCOMA             196
  STATE PRIMARY_WIRE_CENTER_ID  XGS_ENABLED_CNT
0    CO               CLSPCOMA             3475
1    FL               CPCRFLXB            12338
2    CO               DNVRCOCP              150
3    FL               CPCRFLXA               36
4    CO               LTTNCOMA              264


In [6]:
# merge xgs enablement with mg enablement into one dataframe (mg enablement is a subset of xgs enablement)
luqual_enablement_df = df_ora.merge(mg_lu_ora, how = 'left', on = ['STATE','PRIMARY_WIRE_CENTER_ID']).fillna(0)
luqual_enablement_df_save = luqual_enablement_df.copy() #export this as detail

In [7]:
# check data
luqual_enablement_df.head()

,STATE,PRIMARY_WIRE_CENTER_ID,XGS_ENABLED_CNT,MG_ENABLED_CNT
0,CO,CLSPCOMA,3475,0.0
1,FL,CPCRFLXB,12338,0.0
2,CO,DNVRCOCP,150,0.0
3,FL,CPCRFLXA,36,0.0
4,CO,LTTNCOMA,264,0.0


### Merge LU QUAL data with MG Readiness worksheet

In [8]:
wc_mgr_df = pd.read_excel(r'Inputs/111522_OpportunityScore_MultiGig_Readiness.xlsx',sheet_name = 'Datasheet')
wc_mgr_df_save = wc_mgr_df.copy() # save for export in details

In [9]:
wc_mgr_df.columns

Index(['STATE', 'PRIMARY_WIRE_CENTER_ID', 'Opportunity Score', 'CBSA',
       'Core Status', 'Earliest Available For Multi-Gig', 'Cost for Multi-Gig',
       'Aug', 'Sep', 'Oct', 'Nov', 'Dec', '2022 Total', 'Jan', 'Feb', 'Mar',
       'Apr', 'May', 'Jun', 'July', 'Aug.1', 'Sep.1', 'Oct.1', 'Nov.1',
       'Dec.1', '2023 Total', 'Total'],
      dtype='object')

In [10]:
# 10/31/22 - exclude denver cocp wire center site from multigig-candidate

wc_mgr_df = wc_mgr_df[wc_mgr_df["PRIMARY_WIRE_CENTER_ID"] != "DNVRCOCP"]

In [11]:
# obtain only relevant columns
# 10/7/22 - added opportunity score to the data pull
wc_mgr_df = wc_mgr_df[['STATE','PRIMARY_WIRE_CENTER_ID','Core Status','Opportunity Score']]

In [12]:
# clean data - rename columns
wc_mgr_df = wc_mgr_df.rename(columns = {'Core Status': 'CORE_STATUS',
                                        'Opportunity Score': 'OPP_SCORE'})

In [13]:
# merge mg readiness data with lu qual xgs enablement for wc master list
print(wc_mgr_df.head())
print(luqual_enablement_df.head())

xgs_wc_master_df = wc_mgr_df.merge(luqual_enablement_df, how = 'outer', on = ['STATE','PRIMARY_WIRE_CENTER_ID'])

  STATE PRIMARY_WIRE_CENTER_ID    CORE_STATUS  OPP_SCORE
0    UT               STGRUTMA       MultiGIG  12.040135
1    CO               LTTNCOMA  XGS-PON - 180  10.510281
2    FL               STCDFLXA      940M Only   8.930216
3    UT               GTVLUTMA       MultiGIG  18.998338
4    ID               BOISIDMA       MultiGIG  17.283587
  STATE PRIMARY_WIRE_CENTER_ID  XGS_ENABLED_CNT  MG_ENABLED_CNT
0    CO               CLSPCOMA             3475             0.0
1    FL               CPCRFLXB            12338             0.0
2    CO               DNVRCOCP              150             0.0
3    FL               CPCRFLXA               36             0.0
4    CO               LTTNCOMA              264             0.0


In [14]:
# set core status of wire centers with actual MG enablement to MultiGIG, regardless of worksheet status
xgs_wc_master_df.loc[xgs_wc_master_df['MG_ENABLED_CNT']>0,'CORE_STATUS'] = 'MultiGIG'

In [15]:
# base df containing xgs enablement counts, mg enablement counts, and the core network status at wire center level
print(xgs_wc_master_df.shape) # this should pull more wire centers that are not on mg readiness worksheet but in lu_qual_o
xgs_wc_master_df_save = xgs_wc_master_df.copy() #save a copy for export

(218, 6)


### Getting state-level construction funding view

In [16]:
# xgs_wc_master_df as base table = xgs_wc_master_df

# funding information is also at wire center level, join at wire center level then aggregate up

In [17]:
#Import Funding data - export most recent file from PowerBI
Construction_Funding_df = pd.read_excel(r'FundingDetailsExport/Details_Export_Table.xlsx',sheet_name = 'Export')
Construction_Funding_df_save = Construction_Funding_df.copy() # saving a copy for export later

In [18]:
Construction_Funding_df = Construction_Funding_df[['STATE','PRIMARY_WIRE_CENTER_ID','PHASE_ECD_LOGIC','PHASE_NUM_OF_LUS']]
Construction_Funding_df['PHASE_ECD_LOGIC'] = Construction_Funding_df['PHASE_ECD_LOGIC'].dt.to_period('M')
Construction_Funding_df['PHASE_ECD_LOGIC'] = Construction_Funding_df['PHASE_ECD_LOGIC'].astype(str)

In [19]:
Construction_pivot = Construction_Funding_df.groupby(by = ["STATE","PRIMARY_WIRE_CENTER_ID","PHASE_ECD_LOGIC"], as_index = False).sum()
Construction_pivot = Construction_pivot.pivot(index=['STATE','PRIMARY_WIRE_CENTER_ID'], columns='PHASE_ECD_LOGIC')['PHASE_NUM_OF_LUS'].reset_index()

In [20]:
# check data
Construction_pivot = Construction_pivot.fillna(0)
Construction_pivot_save = Construction_pivot.copy() # saving this for export later
Construction_pivot.head()

PHASE_ECD_LOGIC,STATE,PRIMARY_WIRE_CENTER_ID,2022-04,2022-09,2022-10,2022-11,2022-12
0,AZ,TCSNAZFW,0.0,0.0,0.0,160.0,554.0
1,AZ,TCSNAZMA,0.0,0.0,0.0,1485.0,2969.0
2,AZ,TCSNAZRN,0.0,0.0,0.0,564.0,0.0
3,AZ,TCSNAZSW,0.0,0.0,0.0,282.0,479.0
4,CO,CLSPCOMA,0.0,577.0,653.0,0.0,514.0


In [21]:
# clean data - manually adjust field names
Construction_pivot = Construction_pivot.rename(columns = {
                                     '2022-04': '2022APR_CONSTRUCTION_FUNDED',
                                     '2022-09': '2022SEP_CONSTRUCTION_FUNDED',
                                     '2022-10': '2022OCT_CONSTRUCTION_FUNDED',
                                     '2022-11': '2022NOV_CONSTRUCTION_FUNDED',
                                     '2022-12': '2022DEC_CONSTRUCTION_FUNDED'})

In [22]:
# check data
Construction_pivot.head()

PHASE_ECD_LOGIC,STATE,PRIMARY_WIRE_CENTER_ID,2022APR_CONSTRUCTION_FUNDED,2022SEP_CONSTRUCTION_FUNDED,2022OCT_CONSTRUCTION_FUNDED,2022NOV_CONSTRUCTION_FUNDED,2022DEC_CONSTRUCTION_FUNDED
0,AZ,TCSNAZFW,0.0,0.0,0.0,160.0,554.0
1,AZ,TCSNAZMA,0.0,0.0,0.0,1485.0,2969.0
2,AZ,TCSNAZRN,0.0,0.0,0.0,564.0,0.0
3,AZ,TCSNAZSW,0.0,0.0,0.0,282.0,479.0
4,CO,CLSPCOMA,0.0,577.0,653.0,0.0,514.0


In [23]:
# join construction funding to base table
Funded_XGS_WC = xgs_wc_master_df.merge(Construction_pivot, how = 'left', on = ['STATE','PRIMARY_WIRE_CENTER_ID'])

#### View prep

In [24]:
# Creating view

Funded_State_view = Funded_XGS_WC.drop(['OPP_SCORE'], axis = 1).groupby(['STATE'], as_index = False).sum()

# add grand total row - adjust names manually if needed
Funded_State_view.loc[len(Funded_State_view.index)] = ['Grand Total',
                                                       Funded_State_view['XGS_ENABLED_CNT'].sum(),
                                                       Funded_State_view['MG_ENABLED_CNT'].sum(),
                                                       Funded_State_view['2022APR_CONSTRUCTION_FUNDED'].sum(),
                                                       Funded_State_view['2022SEP_CONSTRUCTION_FUNDED'].sum(),
                                                       Funded_State_view['2022OCT_CONSTRUCTION_FUNDED'].sum(),
                                                       Funded_State_view['2022NOV_CONSTRUCTION_FUNDED'].sum(),
                                                       Funded_State_view['2022DEC_CONSTRUCTION_FUNDED'].sum()]

In [25]:
# check data
Funded_State_view.head()

,STATE,XGS_ENABLED_CNT,MG_ENABLED_CNT,2022APR_CONSTRUCTION_FUNDED,2022SEP_CONSTRUCTION_FUNDED,2022OCT_CONSTRUCTION_FUNDED,2022NOV_CONSTRUCTION_FUNDED,2022DEC_CONSTRUCTION_FUNDED
0,AZ,13606.0,10138.0,0.0,0.0,0.0,2491.0,4002.0
1,CO,7598.0,1212.0,0.0,577.0,1702.0,8308.0,7519.0
2,FL,23003.0,0.0,0.0,0.0,3808.0,8550.0,12471.0
3,IA,0.0,0.0,248.0,0.0,0.0,0.0,0.0
4,ID,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# view ready for export 

# adjust names manually if needed
Funded_State_view = Funded_State_view[['STATE','2022APR_CONSTRUCTION_FUNDED','2022SEP_CONSTRUCTION_FUNDED','2022OCT_CONSTRUCTION_FUNDED','2022NOV_CONSTRUCTION_FUNDED','2022DEC_CONSTRUCTION_FUNDED']]
Funded_State_view

,STATE,2022APR_CONSTRUCTION_FUNDED,2022SEP_CONSTRUCTION_FUNDED,2022OCT_CONSTRUCTION_FUNDED,2022NOV_CONSTRUCTION_FUNDED,2022DEC_CONSTRUCTION_FUNDED
0,AZ,0.0,0.0,0.0,2491.0,4002.0
1,CO,0.0,577.0,1702.0,8308.0,7519.0
2,FL,0.0,0.0,3808.0,8550.0,12471.0
3,IA,248.0,0.0,0.0,0.0,0.0
4,ID,0.0,0.0,0.0,0.0,0.0
5,MN,0.0,1130.0,1054.0,1629.0,3235.0
6,NE,0.0,0.0,0.0,0.0,0.0
7,NV,0.0,0.0,0.0,0.0,135.0
8,OR,0.0,76.0,178.0,0.0,0.0
9,UT,0.0,36.0,751.0,3066.0,3621.0


### Getting multigig opportunity summary view

In [27]:
# adjust names manually if needed
MG_Opportunity_Funding = Funded_XGS_WC[['STATE','PRIMARY_WIRE_CENTER_ID','XGS_ENABLED_CNT','MG_ENABLED_CNT','2022NOV_CONSTRUCTION_FUNDED','CORE_STATUS','OPP_SCORE']]

# adding new column that will get construction funding only if the core status is MultiGIG
MG_Opportunity_Funding['2022NOV_CONSTRUCTION_FUNDED_MG'] = np.where(MG_Opportunity_Funding['CORE_STATUS'] == 'MultiGIG',MG_Opportunity_Funding['2022NOV_CONSTRUCTION_FUNDED'],0)

C:\Windows\Temp\ipykernel_12380\3519488978.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  MG_Opportunity_Funding['2022NOV_CONSTRUCTION_FUNDED_MG'] = np.where(MG_Opportunity_Funding['CORE_STATUS'] == 'MultiGIG',MG_Opportunity_Funding['2022NOV_CONSTRUCTION_FUNDED'],0)


In [28]:
# check data

MG_Opportunity_Funding.head()

,STATE,PRIMARY_WIRE_CENTER_ID,XGS_ENABLED_CNT,MG_ENABLED_CNT,2022NOV_CONSTRUCTION_FUNDED,CORE_STATUS,OPP_SCORE,2022NOV_CONSTRUCTION_FUNDED_MG
0,UT,STGRUTMA,2279.0,0.0,2068.0,MultiGIG,12.040135,2068.0
1,CO,LTTNCOMA,264.0,0.0,NaN,XGS-PON - 180,10.510281,0.0
2,FL,STCDFLXA,NaN,NaN,871.0,940M Only,8.930216,0.0
3,UT,GTVLUTMA,NaN,NaN,434.0,MultiGIG,18.998338,434.0
4,ID,BOISIDMA,NaN,NaN,NaN,MultiGIG,17.283587,NaN


In [29]:
MG_Opportunity_Funding_WC_save = MG_Opportunity_Funding.copy() # save copy for export

#### Prep view

In [30]:
# Creating view
MG_Opportunity_Funding_State_view = MG_Opportunity_Funding.groupby(['STATE'], as_index = False)[['XGS_ENABLED_CNT','MG_ENABLED_CNT','2022NOV_CONSTRUCTION_FUNDED','2022NOV_CONSTRUCTION_FUNDED_MG']].sum()

# add grand total row - manually adjust the field names
MG_Opportunity_Funding_State_view.loc[len(MG_Opportunity_Funding_State_view.index)] = ['Grand Total',
                                                       MG_Opportunity_Funding_State_view['XGS_ENABLED_CNT'].sum(),
                                                       MG_Opportunity_Funding_State_view['MG_ENABLED_CNT'].sum(),
                                                       MG_Opportunity_Funding_State_view['2022NOV_CONSTRUCTION_FUNDED'].sum(),
                                                       MG_Opportunity_Funding_State_view['2022NOV_CONSTRUCTION_FUNDED_MG'].sum()]

In [31]:
# check data

MG_Opportunity_Funding_State_view # view ready for export

,STATE,XGS_ENABLED_CNT,MG_ENABLED_CNT,2022NOV_CONSTRUCTION_FUNDED,2022NOV_CONSTRUCTION_FUNDED_MG
0,AZ,13606.0,10138.0,2491.0,2331.0
1,CO,7598.0,1212.0,8308.0,6826.0
2,FL,23003.0,0.0,8550.0,0.0
3,IA,0.0,0.0,0.0,0.0
4,ID,0.0,0.0,0.0,0.0
5,MN,8418.0,8353.0,1629.0,1349.0
6,NE,0.0,0.0,0.0,0.0
7,NV,0.0,0.0,0.0,0.0
8,OR,349.0,264.0,0.0,0.0
9,UT,3080.0,241.0,3066.0,3066.0


### Getting Subs views

Getting current MG opportunity

In [32]:
# create original view but with all MG wire centers from above base
# active subs y/n -- pending subs y/n -- earliest due date for pending subs

# create sub view of above with only wire centers in states that have at least one mg enablement
# graph to illustrate MG opportunity

In [33]:
# start with same base - check data
xgs_wc_master_df.head()

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT
0,UT,STGRUTMA,MultiGIG,12.040135,2279.0,0.0
1,CO,LTTNCOMA,XGS-PON - 180,10.510281,264.0,0.0
2,FL,STCDFLXA,940M Only,8.930216,NaN,NaN
3,UT,GTVLUTMA,MultiGIG,18.998338,NaN,NaN
4,ID,BOISIDMA,MultiGIG,17.283587,NaN,NaN


In [34]:
# fillna

xgs_wc_master_df[['XGS_ENABLED_CNT','MG_ENABLED_CNT']] = xgs_wc_master_df[['XGS_ENABLED_CNT','MG_ENABLED_CNT']].fillna(0)

In [35]:
# derive MG untapped column
xgs_wc_master_df['MG_UNTAPPED'] = xgs_wc_master_df['XGS_ENABLED_CNT'] - xgs_wc_master_df['MG_ENABLED_CNT']

In [36]:
# check data
xgs_wc_master_df.head()

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED
0,UT,STGRUTMA,MultiGIG,12.040135,2279.0,0.0,2279.0
1,CO,LTTNCOMA,XGS-PON - 180,10.510281,264.0,0.0,264.0
2,FL,STCDFLXA,940M Only,8.930216,0.0,0.0,0.0
3,UT,GTVLUTMA,MultiGIG,18.998338,0.0,0.0,0.0
4,ID,BOISIDMA,MultiGIG,17.283587,0.0,0.0,0.0


In [37]:
# import salesforce summary to get active/pending subs
sf_df_sum = pd.read_excel(r'Salesforce Data/Salesforce_Summary.xlsx',sheet_name = 'Summary')
sf_df_details = pd.read_excel(r'Salesforce Data/Salesforce_Summary.xlsx',sheet_name = 'Details') 

In [38]:
# get relevant columns in salesforce summary df
sf_df_sum = sf_df_sum[['WIRE_CENTER_ID','Total Active Subscribers','Total Pending Activation Subscribers']].rename(columns = {'WIRE_CENTER_ID':'PRIMARY_WIRE_CENTER_ID'})

In [39]:
# wire centers with active subs
sf_df_sum = sf_df_sum[(sf_df_sum['Total Active Subscribers']>0)|(sf_df_sum['Total Pending Activation Subscribers']>0)]

In [40]:
# check data
sf_df_sum

,PRIMARY_WIRE_CENTER_ID,Total Active Subscribers,Total Pending Activation Subscribers
0,0,1,72
392,PHNXAZNE,12,1
393,PHNXAZNO,0,1
394,PHNXAZSO,30,3
396,SPRSAZEA,33,1
397,SPRSAZMA,16,1
398,TCSNAZCR,16,0
399,TCSNAZFW,5,5
400,TCSNAZMA,81,22
401,TCSNAZRN,1,3


In [41]:
# get earliest pending activation due date

from datetime import timedelta, date
from pandas.tseries.offsets import DateOffset

yesterday_date = date.today() - timedelta(days=90)
future_date = date.today() + timedelta(days=90)

duedate_mask = (sf_df_details['Account Status']=='Pending Activation') & (pd.to_datetime(sf_df_details['Due Date']).dt.date > yesterday_date) & (pd.to_datetime(sf_df_details['Due Date']).dt.date <= future_date)
sf_df_duedates = sf_df_details.loc[duedate_mask,['Wire Center','Due Date']]

In [42]:
# update Due Date field to date only
sf_df_duedates['Due Date'] = pd.to_datetime(sf_df_duedates['Due Date']).dt.date

In [43]:
# get earliest due date by wire center

sf_df_duedates = sf_df_duedates.groupby(['Wire Center'], as_index = False).agg(np.min).rename(columns = {'Wire Center':'PRIMARY_WIRE_CENTER_ID',
                                                                                                         'Due Date': 'Earliest Due Date'})

In [44]:
# join due dates to summary

sf_df_sum = sf_df_sum.merge(sf_df_duedates, on = 'PRIMARY_WIRE_CENTER_ID', how = 'left')

In [45]:
# join salesforce to xgs_wc_master_df

subs_view = xgs_wc_master_df.merge(sf_df_sum, how = 'left', on = 'PRIMARY_WIRE_CENTER_ID')

#### Prep view

In [46]:
# check data
subs_view.head()

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED,Total Active Subscribers,Total Pending Activation Subscribers,Earliest Due Date
0,UT,STGRUTMA,MultiGIG,12.040135,2279.0,0.0,2279.0,22.0,22.0,2022-11-14
1,CO,LTTNCOMA,XGS-PON - 180,10.510281,264.0,0.0,264.0,4.0,0.0,NaN
2,FL,STCDFLXA,940M Only,8.930216,0.0,0.0,0.0,NaN,NaN,NaN
3,UT,GTVLUTMA,MultiGIG,18.998338,0.0,0.0,0.0,NaN,NaN,NaN
4,ID,BOISIDMA,MultiGIG,17.283587,0.0,0.0,0.0,NaN,NaN,NaN


In [47]:
# update subs columns to Y/N

subs_view['Active Subs Y/N'] = np.where(subs_view['Total Active Subscribers'] > 0, 'Y', 'N')
subs_view['Pending Subs Y/N'] = np.where(subs_view['Total Pending Activation Subscribers'] > 0, 'Y', 'N')

In [48]:
subs_view_save = subs_view.copy() # save copy for export

In [49]:
# drop columns
subs_view = subs_view.drop(['Total Active Subscribers','Total Pending Activation Subscribers'], axis = 1)

In [50]:
# reorder columns

subs_view = subs_view[['STATE','PRIMARY_WIRE_CENTER_ID','CORE_STATUS','OPP_SCORE','XGS_ENABLED_CNT','MG_ENABLED_CNT','MG_UNTAPPED','Active Subs Y/N','Pending Subs Y/N','Earliest Due Date']]

In [51]:
# only wire centers with XGS enablement

subs_view = subs_view[subs_view['XGS_ENABLED_CNT'] > 0]

In [52]:
# ready for export

subs_view

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED,Active Subs Y/N,Pending Subs Y/N,Earliest Due Date
0,UT,STGRUTMA,MultiGIG,12.040135,2279.0,0.0,2279.0,Y,Y,2022-11-14
1,CO,LTTNCOMA,XGS-PON - 180,10.510281,264.0,0.0,264.0,Y,N,NaN
10,UT,MAGNUTNM,MultiGIG,15.255279,295.0,0.0,295.0,Y,N,NaN
13,CO,SCRTCOMA,MultiGIG,14.355833,94.0,0.0,94.0,Y,N,NaN
21,CO,DNVRCOMA,MultiGIG,13.735440,438.0,438.0,0.0,Y,Y,2022-10-15
22,MN,MPLSMNGE,MultiGIG,13.691429,1772.0,1772.0,0.0,Y,Y,2022-11-29
26,CO,CLSPCOMA,MultiGIG,13.343808,3475.0,0.0,3475.0,Y,Y,2022-11-21
28,MN,CRYSMNCR,MultiGIG,13.288129,978.0,978.0,0.0,Y,Y,2022-12-15
29,OR,MLWKOR17,MultiGIG,13.242958,85.0,0.0,85.0,N,N,NaN
34,AZ,PHNXAZNE,MultiGIG,12.720800,210.0,210.0,0.0,Y,Y,2022-11-30


### MG only subs view

In [53]:
# start with subs view
mg_subs_view = subs_view.copy()

In [54]:
# getting only Core Network Status MultiGIG
mg_subs_view = mg_subs_view[mg_subs_view['CORE_STATUS'] == 'MultiGIG'].sort_values(by = ['STATE','PRIMARY_WIRE_CENTER_ID'])

In [55]:
# check data
mg_subs_view

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED,Active Subs Y/N,Pending Subs Y/N,Earliest Due Date
34,AZ,PHNXAZNE,MultiGIG,12.720800,210.0,210.0,0.0,Y,Y,2022-11-30
47,AZ,PHNXAZSO,MultiGIG,11.211576,1325.0,1325.0,0.0,Y,Y,2022-11-29
91,AZ,TCSNAZCR,MultiGIG,8.356103,400.0,400.0,0.0,Y,N,NaN
100,AZ,TCSNAZMA,MultiGIG,7.669243,9205.0,8203.0,1002.0,Y,Y,2022-11-23
99,AZ,TCSNAZRN,MultiGIG,7.889057,631.0,0.0,631.0,Y,Y,2022-11-10
82,AZ,TCSNAZSW,MultiGIG,8.926796,445.0,0.0,445.0,N,N,NaN
52,CO,ARVDCOMA,MultiGIG,10.753984,273.0,196.0,77.0,Y,N,2022-12-09
26,CO,CLSPCOMA,MultiGIG,13.343808,3475.0,0.0,3475.0,Y,Y,2022-11-21
63,CO,CLSPCOPV,MultiGIG,10.183161,2256.0,0.0,2256.0,Y,Y,2022-11-17
69,CO,DNVRCODC,MultiGIG,9.853355,166.0,166.0,0.0,Y,N,NaN


In [56]:
# get data with non-zero untapped MG

mg_subs_untapped_view = mg_subs_view[mg_subs_view['MG_UNTAPPED'] > 0]

In [57]:
# ready for export

mg_subs_untapped_view

,STATE,PRIMARY_WIRE_CENTER_ID,CORE_STATUS,OPP_SCORE,XGS_ENABLED_CNT,MG_ENABLED_CNT,MG_UNTAPPED,Active Subs Y/N,Pending Subs Y/N,Earliest Due Date
100,AZ,TCSNAZMA,MultiGIG,7.669243,9205.0,8203.0,1002.0,Y,Y,2022-11-23
99,AZ,TCSNAZRN,MultiGIG,7.889057,631.0,0.0,631.0,Y,Y,2022-11-10
82,AZ,TCSNAZSW,MultiGIG,8.926796,445.0,0.0,445.0,N,N,NaN
52,CO,ARVDCOMA,MultiGIG,10.753984,273.0,196.0,77.0,Y,N,2022-12-09
26,CO,CLSPCOMA,MultiGIG,13.343808,3475.0,0.0,3475.0,Y,Y,2022-11-21
63,CO,CLSPCOPV,MultiGIG,10.183161,2256.0,0.0,2256.0,Y,Y,2022-11-17
13,CO,SCRTCOMA,MultiGIG,14.355833,94.0,0.0,94.0,Y,N,NaN
29,OR,MLWKOR17,MultiGIG,13.242958,85.0,0.0,85.0,N,N,NaN
10,UT,MAGNUTNM,MultiGIG,15.255279,295.0,0.0,295.0,Y,N,NaN
0,UT,STGRUTMA,MultiGIG,12.040135,2279.0,0.0,2279.0,Y,Y,2022-11-14


### New tab: Internal document - polygon info

In [58]:
# exclude existing polygons that are mg enabled. Get opportunity scores

In [59]:
# using mg_subs_untapped_view as base, create polygon info sheet
mg_info_view = mg_subs_untapped_view.copy()

In [60]:
# get relevant polygon info at wire center level
polygon_info = df_ora_details[['PRIMARY_WIRE_CENTER_ID','UNI_NDSJOBNUMBER','UNI_POLYGONID', 'OLT_NET_ID']]

In [61]:
# keep unique values
polygon_info = polygon_info.drop_duplicates(ignore_index=True)

In [62]:
# set OLT_NET_ID NaN to "none"
polygon_info['OLT_NET_ID'] = polygon_info['OLT_NET_ID'].fillna("")

In [63]:
polygon_info = polygon_info.groupby(['PRIMARY_WIRE_CENTER_ID','UNI_POLYGONID']) \
   .agg({'UNI_NDSJOBNUMBER' : lambda x: ','.join(x.unique()),
         'OLT_NET_ID' : ','.join}) \
   .reset_index() \
   .reindex(columns=polygon_info.columns)

In [64]:
# join polygon info to mg view

mg_info_view = mg_info_view[['STATE','PRIMARY_WIRE_CENTER_ID','OPP_SCORE']].merge(polygon_info, how = 'left', on = 'PRIMARY_WIRE_CENTER_ID')

In [65]:
# join to salesforce data at polygon id level
# show only salesforce data that has active subscribers and pending subscribers within 3 months in future
duedate_mask = (sf_df_details['Account Status']=='Pending Activation') & (pd.to_datetime(sf_df_details['Due Date']).dt.date >= date.today()) & (pd.to_datetime(sf_df_details['Due Date']).dt.date <= future_date)
sf_df_details_polygon = sf_df_details[(sf_df_details['Account Status'] == 'Active') | duedate_mask]

In [66]:
sf_df_details_polygon = sf_df_details_polygon[['Polygon ID','Account Status']].groupby(['Polygon ID','Account Status'], as_index = False).size().pivot(index = 'Polygon ID', columns = 'Account Status', values = 'size').reset_index().fillna(0)


In [67]:
# update salesforce polygon column to object
sf_df_details_polygon['Polygon ID'] = sf_df_details_polygon['Polygon ID'].astype(str)

In [68]:
mg_info_view = mg_info_view.rename(columns = {'UNI_POLYGONID': 'Polygon ID'})
mg_info_view = mg_info_view.merge(sf_df_details_polygon[['Polygon ID','Active','Pending Activation']], on = 'Polygon ID', how = 'left').fillna(0)

In [69]:
mg_info_view

,STATE,PRIMARY_WIRE_CENTER_ID,OPP_SCORE,UNI_NDSJOBNUMBER,Polygon ID,OLT_NET_ID,Active,Pending Activation
0,AZ,TCSNAZMA,7.669243,N.957868,1179639,",TCSNAZMAOL6",22.0,6.0
1,AZ,TCSNAZMA,7.669243,N.957886,1179689,"TCSNAZMAOL5,",9.0,5.0
2,AZ,TCSNAZMA,7.669243,N.916206,1179690,",TCSNAZMAOL7",8.0,0.0
3,AZ,TCSNAZMA,7.669243,N.957879,1179700,",TCSNAZMAOL7",4.0,0.0
4,AZ,TCSNAZMA,7.669243,N.927853,1179701,",TCSNAZMAOL6",1.0,0.0
...,...,...,...,...,...,...,...,...
78,WA,STTLWA04,10.624742,N.910868,1180355,,0.0,1.0
79,WA,STTLWA05,10.561038,N.927902,1180117,",STTLWA05OL5",0.0,3.0
80,WA,STTLWASU,11.683616,N.916245,1180086,",STTLWASUOL7",4.0,0.0
81,WA,STTLWASU,11.683616,N.902454,1180111,",STTLWASUOL7",10.0,4.0


In [70]:
# reorder columns, rename columns
mg_info_view = mg_info_view[['STATE','PRIMARY_WIRE_CENTER_ID','Polygon ID','UNI_NDSJOBNUMBER','OLT_NET_ID','Active','Pending Activation']]

In [71]:
mg_info_view = mg_info_view.rename(columns = {'Polygon ID':'POLYGON_ID',
                               'Active': 'ACTIVE_SUBS_CNT',
                               'Pending Activation': 'PENDING_SUBS_CNT'})

mg_info_view.head()

,STATE,PRIMARY_WIRE_CENTER_ID,POLYGON_ID,UNI_NDSJOBNUMBER,OLT_NET_ID,ACTIVE_SUBS_CNT,PENDING_SUBS_CNT
0,AZ,TCSNAZMA,1179639,N.957868,",TCSNAZMAOL6",22.0,6.0
1,AZ,TCSNAZMA,1179689,N.957886,"TCSNAZMAOL5,",9.0,5.0
2,AZ,TCSNAZMA,1179690,N.916206,",TCSNAZMAOL7",8.0,0.0
3,AZ,TCSNAZMA,1179700,N.957879,",TCSNAZMAOL7",4.0,0.0
4,AZ,TCSNAZMA,1179701,N.927853,",TCSNAZMAOL6",1.0,0.0


In [72]:
# join in xgs enablement and mg enablement info

# XGS qualifications count at polygon level

polygon_xgs_query = """
            WITH QD AS (
            SELECT  
                    O.LUID --count distinct LU id's
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
                    ,count(distinct LUID) as xgs_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
            

                                """


polygon_xgs_enablement_df = pd.read_sql(polygon_xgs_query, con=conn)

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [73]:
# mg enablement at polygon level
polygon_multigig_query = """

            WITH QD AS (
            SELECT  
                    O.LUID
                    ,O.STATE
                    ,O.WIRE_CENTER_ID AS PRIMARY_WIRE_CENTER_ID
                    ,O.ADDRESS
                    ,O.UNI_SERVICETYPE
                    ,O.OLT
                    ,O.TECH_TYPE
                    ,O.OLT_TYPE
                    ,O.FSAI
                    ,O.UNI_POLYGONID
                    ,O.UNI_STATUS
                    ,O.UNI_NDSJOBNUMBER
                    ,O.UNI_FWRKSJOBNUMBER
                    --,O.*
                    
            FROM    LQSMO.LU_QUAL_O O
            
            WHERE   ADDRESS <> '7599 S WHEELING CT'
                    AND (UNI_SERVICETYPE LIKE 'XGS%' 
                        OR TECH_TYPE LIKE 'XGS_PON' 
                        OR OLT_TYPE='ADTRAN-SDX6320-16-XGS-PON') 
                    AND uni_status='IN SERVICE'
                    AND UNI_POLYGONID in ('1179499',  -- PHNXAZNE
                                          '1179547', '1179498', '1179121','1180067','1180494', -- PHNXAZSO
                                          '1178832', -- ARVDCOMA
                                          '1179849','1180344','1180107','1180306','1180028', '1178694', '1179569',-- MPLSMNBE 
                                          '1179862', '1179836', '1179604', '1179838', '1179887', '1180054', '1180222', -- MPLSMNBE
                                          '1178144', --PYLPWA01
                                          '1179687', '1179674','1180254', --GLDNCOMA
                                          '1180012','1180391', -- PTLDOR11
                                          '1179864', -- DNVRCOMA 
                                          '1179553','1179924', -- STPLMNBE 
                                          '1179688','1179157', '1179574',  -- MPLSMNTF
                                          '1180026','1180288','1179808','1179976', -- MPLSMNGE
                                          '1179583', -- DNVRCODC
                                          '1179774','1180265','1180031','1180237', -- CRYSMNCR
                                          '1180315','1189074', -- MRRYUTMA
                                          '1179708','1179714','1179719','1180072','1180075','1180089','1180095', -- TCSNAZMA
                                          '1180096','1179639','1179689','1179690','1179700','1179701','1179702', -- TCSNAZMA
                                          '1179703','1179704','1179718','1179793','1179803','1180092','550715', -- TCSNAZMA
                                          '1180094','1180507', --TCSNAZMA
                                          '1179641','1179914' --TCSNAZCR
                                          ) --add in MG polygon IDs here
                    
            )
            
            SELECT  STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID
                    ,count(distinct LUID) as mg_enabled_cnt
                    
            FROM    QD
            
            GROUP BY
                    STATE
                    ,PRIMARY_WIRE_CENTER_ID
                    ,UNI_POLYGONID


"""

polygon_multigig_enablement_df = pd.read_sql(polygon_multigig_query, con=conn) #contains total lu at wc level

C:\Users\AD21236\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [74]:
# check data
print(polygon_multigig_enablement_df.head())
print(polygon_xgs_enablement_df.head())

  STATE PRIMARY_WIRE_CENTER_ID UNI_POLYGONID  MG_ENABLED_CNT
0    MN               CRYSMNCR       1180265             140
1    CO               GLDNCOMA       1179674             158
2    MN               MPLSMNTF       1179157             365
3    AZ               TCSNAZMA       1180072             325
4    AZ               TCSNAZMA       1179639             420
  STATE PRIMARY_WIRE_CENTER_ID UNI_POLYGONID  XGS_ENABLED_CNT
0    CO               CLSPCOMA       1179971              150
1    CO               CLSPCOPV       1179966              231
2    FL               CPCRFLXB       1179775              448
3    FL               CPCRFLXB       1180058              122
4    FL               CPCRFLXB       1180212              666


In [75]:
polygon_enablement_mg_xgs = polygon_xgs_enablement_df.merge(polygon_multigig_enablement_df, how = 'left', on = ['STATE','PRIMARY_WIRE_CENTER_ID','UNI_POLYGONID']).fillna(0)

In [76]:
polygon_enablement_mg_xgs = polygon_enablement_mg_xgs.rename(columns = {'UNI_POLYGONID': 'POLYGON_ID'})

In [77]:
mg_info_view = mg_info_view.merge(polygon_enablement_mg_xgs, how = 'left', on = ['STATE','PRIMARY_WIRE_CENTER_ID','POLYGON_ID'])

In [78]:
mg_info_view['MG_UNTAPPED_LU'] = mg_info_view['XGS_ENABLED_CNT'] - mg_info_view['MG_ENABLED_CNT']

In [79]:
# export

import time

TodaysDate = time.strftime("%m%d%y")
excelfilename = "Output/" + TodaysDate +"_Multigig_Enablement_Scorecard_Report_New_Core_Status.xlsx"

with pd.ExcelWriter(excelfilename) as writer:
    mg_subs_untapped_view.to_excel(writer, sheet_name='MG_Subs_Untapped_Summary', index = False)
    subs_view.to_excel(writer, sheet_name='XGS_Subs_Summary', index = False)
    MG_Opportunity_Funding_State_view.to_excel(writer, sheet_name='MG_Enablement_Tracker', index = False)
    Funded_State_view.to_excel(writer, sheet_name='Construction_Funded_Outlook', index = False)
    mg_info_view.to_excel(writer, sheet_name='Polygon_info', index = False)
 
    
    # raw data
    df_ora_details.to_excel(writer, sheet_name='LUQUALO_Details', index = False)
    Construction_Funding_df_save.to_excel(writer, sheet_name='XGSPrep_Details_PowerBI', index = False)
    wc_mgr_df_save.to_excel(writer, sheet_name='MG_Readiness_details', index = False)
    


In [5]:
#create ad-hoc directory
directory = "Output/" + TodaysDate + " ad-hoc views"
os.makedirs(directory)